In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Define the autoencoder model
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim * 2) # output both mean and std deviation
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x):
        # encode input
        # print(x.shape)
        z_mean_logvar = self.encoder(x)
        z_mean = z_mean_logvar[:, :latent_dim]
        z_logvar = z_mean_logvar[:, latent_dim:]
        # reparameterize
        z = self.reparameterize(z_mean, z_logvar)
        # decode latent representation
        x_hat = self.decoder(z)
        return x_hat, z_mean, z_logvar


# Define the loss function (reconstruction error + KL divergence)
def loss_function(x, x_hat, z_mean, z_logvar):
    recon_loss = nn.MSELoss()(x_hat, x)
    kl_div = -0.5 * torch.sum(1 + z_logvar - z_mean.pow(2) - z_logvar.exp())
    return recon_loss + kl_div, recon_loss

def train_ae(model, train_loader, val_loader, num_epochs, lr):

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    model.to(device)
    
    # Define the optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    # criterion = nn.MSELoss()

    # Set up the training and validation loops
    best_val_loss = float("inf")
    for epoch in range(num_epochs):
        train_loss = 0.0
        val_loss = 0.0
        
        # Train loop
        model.train()
        for i, data in enumerate(train_loader):
            # Get the input data
            inputs, _ = data
            inputs = inputs.to(device)
            
            # Forward pass
            recon, mu, logvar = model(inputs)
            loss = loss_function(recon, inputs, mu, logvar)[0]
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        # Validation loop
        model.eval()
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                # Get the input data
                inputs, _ = data
                inputs = inputs.to(device)
                
                # Forward pass
                recon, mu, logvar = model(inputs)
                loss = loss_function(recon, inputs, mu, logvar)[1]
                
                val_loss += loss.item()
                
        # Compute average losses
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")
    
        # Save the model if it performs better on validation set
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "new_best_model.pt")

# Use the trained encoder to reduce dimensionality of feature vectors
def reduce_dim(model, x):
    with torch.no_grad():
        x = torch.FloatTensor(x)
        # z_mean, _ = model.encoder(x).chunk(2, dim=1)
        z_mean, _ = model(x).chunk(2, dim=1)
    return z_mean.numpy() # return latent representation as numpy array

In [33]:
# Create the autoencoder and optimizer

# Define training hyperparameters
input_dim = 1024
hidden_dim = 1024
latent_dim = 1024
lr = 1e-5
batch_size = 25
num_epochs = 100

# device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = VAE(input_dim, hidden_dim, latent_dim)#.to(device=device)
optimizer = optim.Adam(model.parameters(), lr=lr)

# Load your data into PyTorch tensors and create a DataLoader
raw = np.load('3rd_layer_WideResNet.npy')
# print(raw.shape)
raw_shaped = raw.reshape(-1, raw.shape[1])
# print(raw_shaped.shape)
test_selected = np.random.choice(raw_shaped.shape[0], size = int(raw_shaped.shape[0]*0.1), replace = False)
train_selected = [el for el in range(raw_shaped.shape[0]) if el not in test_selected]
train_data_raw = raw_shaped[train_selected]
val_data_raw = raw_shaped[test_selected]
# train_data = torch.from_numpy(raw_shaped)
train_dataset = TensorDataset(torch.from_numpy(train_data_raw), torch.from_numpy(train_data_raw))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)#.to(device=device)
val_dataset = TensorDataset(torch.from_numpy(val_data_raw), torch.from_numpy(val_data_raw))
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)#.to(device=device)

train_ae(model=model, train_loader=train_loader, val_loader=val_loader, num_epochs=num_epochs, lr=lr)

In [28]:
loaded_model = VAE(input_dim, hidden_dim, latent_dim)
state_dict = torch.load('new_best_model.pt')
loaded_model.load_state_dict(state_dict)

RuntimeError: Error(s) in loading state_dict for VAE:
	size mismatch for encoder.0.weight: copying a param with shape torch.Size([128, 1024]) from checkpoint, the shape in current model is torch.Size([1024, 1024]).
	size mismatch for encoder.0.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([1024]).
	size mismatch for encoder.2.weight: copying a param with shape torch.Size([64, 128]) from checkpoint, the shape in current model is torch.Size([2048, 1024]).
	size mismatch for encoder.2.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([2048]).
	size mismatch for decoder.0.weight: copying a param with shape torch.Size([128, 32]) from checkpoint, the shape in current model is torch.Size([1024, 1024]).
	size mismatch for decoder.0.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([1024]).
	size mismatch for decoder.

In [24]:
reduce_dim(model=loaded_model, x=raw_shaped[1231])

AttributeError: 'collections.OrderedDict' object has no attribute 'encoder'

In [6]:
b = torch.hub.load('pytorch/vision:v0.9.0', 'wide_resnet50_2', pretrained=True)

Using cache found in /home/jo/.cache/torch/hub/pytorch_vision_v0.9.0


In [11]:
for k, el in enumerate(b.children()):
    print(k)
    print(el)

0
Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
1
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
2
ReLU(inplace=True)
3
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
4
Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1

In [13]:
import numpy as np

In [16]:
c = torch.randn((1000, input_dim)).numpy()

In [19]:
features = np.load('3rd_layer_WideResNet.npy')

In [20]:
features.shape

(453, 1024, 14, 14)

In [30]:
ae_train_3rd = features.reshape(-1, features.shape[-3])#, features.shape[3])

In [31]:
ae_train_3rd.shape

(88788, 1024)

In [12]:
raw = np.load('3rd_layer_WideResNet.npy')
print(raw.shape)
raw_shaped = raw.reshape(-1, raw.shape[1])
print(raw_shaped.shape)

(453, 1024, 14, 14)
(88788, 1024)


In [23]:
train_data_raw.shape

(71031, 1024)

In [18]:
val_data_raw.shape

(17757, 1024)